# Chapter 5: Methods – The Building Blocks of Logic

As your programs grow in complexity, you'll find yourself writing the same code repeatedly or needing to organize your logic into self‑contained, reusable units. This is where **methods** come in. A method is a block of code that performs a specific task and can be called from other parts of your program. Methods are the primary way to structure code, promote reuse, and make your programs more maintainable.

In this chapter, you'll learn everything you need to know about methods in C#:

- **Method signatures** – the combination of name, parameters, and return type that defines a method.
- **Parameters** – how to pass data into a method, including advanced techniques like `ref`, `out`, `in`, and parameter arrays (`params`).
- **Method overloading** – creating multiple methods with the same name but different parameters.
- **Local functions** – defining methods inside other methods for better encapsulation.
- **Expression‑bodied members** – a concise syntax for simple methods and properties.

By the end, you'll be able to design clean, reusable methods that form the backbone of well‑structured applications.

---

## 5.1 Anatomy of a Method – Understanding Method Signatures

Every method in C# has a **signature** that uniquely identifies it within a class. The signature consists of:

- The **method name** (must be a valid identifier).
- The **number, types, and order of its parameters** (but not the parameter names).
- The **type of the value it returns** (not part of the signature for overloading, but important for the method definition).
- **Access modifiers** (like `public`, `private`) are part of the method declaration but not part of the signature used for overloading.

Here's a typical method declaration:

```csharp
public int Add(int a, int b)
{
    return a + b;
}
```

Let's break it down:

- `public` – an **access modifier** that determines where the method can be called from. `public` means any code can call it.
- `int` – the **return type**. This method returns an integer.
- `Add` – the **method name**.
- `(int a, int b)` – the **parameter list**. This method takes two integers, named `a` and `b`.
- `{ ... }` – the **method body**, containing the code that executes when the method is called.

### Return Types

A method can return a value of any type, or it can return nothing by specifying `void`.

```csharp
public void PrintMessage(string message)
{
    Console.WriteLine(message);
    // no return statement needed (or you can use "return;" to exit early)
}
```

If the return type is not `void`, the method **must** have a `return` statement that returns a value convertible to that type.

### Parameters

Parameters are variables that receive values when the method is called. They define what data the method needs to do its job.

```csharp
public double CalculateCircleArea(double radius)
{
    return Math.PI * radius * radius;
}
```

When calling this method, you provide an argument for each parameter:

```csharp
double area = CalculateCircleArea(5.0);
```

### Access Modifiers

The most common access modifiers are:

- `public` – accessible from anywhere.
- `private` – accessible only within the same class (default if no modifier is specified).
- `internal` – accessible only within the same assembly (project).
- `protected` – accessible within the same class and derived classes.

Choosing the right access level is part of encapsulation – hide internal details, expose only what's necessary.

### Static vs. Instance Methods

Methods can be **static** or **instance**:

- **Instance methods** belong to an instance of a class. You need an object to call them.
- **Static methods** belong to the class itself. You call them using the class name.

```csharp
public class Calculator
{
    // Instance method
    public int AddInstance(int a, int b) => a + b;

    // Static method
    public static int AddStatic(int a, int b) => a + b;
}

// Usage
Calculator calc = new Calculator();
int result1 = calc.AddInstance(3, 4);      // call on instance

int result2 = Calculator.AddStatic(3, 4);  // call on class
```

`Console.WriteLine` and `Math.Pow` are examples of static methods. Use static methods for functionality that doesn't depend on object state.

---

## 5.2 Parameters Deep Dive: `ref`, `out`, `in`, and `params`

By default, arguments are passed **by value**. For value types, this means a copy of the value is passed; for reference types, a copy of the reference is passed (so the method can modify the object, but cannot reassign the reference to point to a new object). C# provides keywords to modify this behavior.

### `ref` – Pass by Reference

The `ref` keyword causes arguments to be passed **by reference**. Changes made to the parameter inside the method are reflected in the original variable. This works for both value types and reference types.

```csharp
public void Swap(ref int x, ref int y)
{
    int temp = x;
    x = y;
    y = temp;
}

// Usage
int a = 5, b = 10;
Swap(ref a, ref b);
Console.WriteLine($"a = {a}, b = {b}"); // a = 10, b = 5
```

When using `ref`, the argument **must be initialized** before being passed. The method can read and modify it.

### `out` – Output Parameters

The `out` keyword is similar to `ref`, but it's used when a method needs to return multiple values. The parameter does not need to be initialized before calling; it must be assigned a value inside the method before the method returns.

```csharp
public bool TryParseInt(string input, out int result)
{
    return int.TryParse(input, out result);
}

// Usage
if (TryParseInt("123", out int number))
{
    Console.WriteLine($"Parsed: {number}");
}
else
{
    Console.WriteLine("Parse failed");
}
```

This pattern is common in .NET (e.g., `int.TryParse`). The `out` variable can be declared inline (as shown) – a feature introduced in C# 7.0.

### `in` – Readonly Reference

The `in` keyword passes an argument by reference but guarantees that the method will not modify it. It's primarily used for performance with large structs to avoid copying, while ensuring immutability.

```csharp
public void Print(in Point p)
{
    // p.X = 10; // Compiler error – cannot modify
    Console.WriteLine($"({p.X}, {p.Y})");
}
```

The caller doesn't need to use `in` explicitly (though they can). The compiler enforces that the method does not write to the parameter.

### Parameter Arrays (`params`)

Sometimes you want a method to accept a variable number of arguments. The `params` keyword allows you to pass a comma‑separated list of values, which the method receives as an array.

```csharp
public int Sum(params int[] numbers)
{
    int total = 0;
    foreach (int n in numbers)
    {
        total += n;
    }
    return total;
}

// Usage
int sum1 = Sum(1, 2, 3);           // 6
int sum2 = Sum(1, 2, 3, 4, 5);     // 15
int sum3 = Sum();                   // 0 (empty array)
int sum4 = Sum(new int[] { 10, 20 }); // you can also pass an array directly
```

Rules for `params`:

- Only one `params` parameter per method.
- It must be the last parameter in the parameter list.
- The parameter must be a single‑dimensional array.

`Console.WriteLine` uses `params` for its formatting arguments: `WriteLine(string format, params object[] args)`.

### Named and Optional Arguments

C# also supports **named arguments** and **optional parameters**, which can make method calls more readable and flexible.

#### Optional Parameters

You can specify default values for parameters. If the caller omits that argument, the default value is used.

```csharp
public void DisplayInfo(string name, int age = 0, string city = "Unknown")
{
    Console.WriteLine($"{name}, Age: {age}, City: {city}");
}

// Usage
DisplayInfo("Alice", 30, "New York");
DisplayInfo("Bob", 25);          // city defaults to "Unknown"
DisplayInfo("Charlie");          // age defaults to 0, city defaults to "Unknown"
```

Optional parameters must appear after all required parameters.

#### Named Arguments

When calling a method, you can specify which parameter each argument belongs to by using the parameter name followed by a colon. This allows you to skip optional parameters or reorder arguments.

```csharp
DisplayInfo(city: "London", name: "David", age: 40);
```

Named arguments are especially useful when a method has many optional parameters and you only want to specify a few.

---

## 5.3 Method Overloading

**Method overloading** allows a class to have multiple methods with the same name, as long as their signatures (parameter count, types, or order) are different. The return type alone is not enough to distinguish overloads.

```csharp
public class Printer
{
    public void Print(string message)
    {
        Console.WriteLine(message);
    }

    public void Print(int number)
    {
        Console.WriteLine(number);
    }

    public void Print(string message, int times)
    {
        for (int i = 0; i < times; i++)
        {
            Console.WriteLine(message);
        }
    }
}
```

The compiler decides which overload to call based on the arguments provided at the call site.

```csharp
Printer p = new Printer();
p.Print("Hello");          // calls first overload
p.Print(42);               // calls second overload
p.Print("Hi", 3);          // calls third overload
```

Overloading is commonly used to provide convenience methods or to handle different data types. LINQ is full of overloads (e.g., `Where` has several).

### Overloading vs. Optional Parameters

Both can achieve similar results. A rule of thumb: use optional parameters when the parameter has a natural default, and overloading when the method's behavior is qualitatively different (e.g., one version accepts a string, another accepts a number). Overloading can also be used when default values would cause ambiguity or when you need different parameter names.

---

## 5.4 Local Functions

Introduced in C# 7.0, **local functions** are methods nested inside another method. They are private to the containing method and can be useful for encapsulating helper logic that only makes sense within that scope.

```csharp
public int CalculateFactorial(int n)
{
    if (n < 0) throw new ArgumentException("n must be non‑negative");

    // Local function
    int Factorial(int x)
    {
        if (x <= 1) return 1;
        return x * Factorial(x - 1);
    }

    return Factorial(n);
}
```

Local functions can access variables from the containing method (like `n`), which can be convenient but also a source of subtle bugs. They support recursion, generics, and even async/await.

Benefits:

- **Encapsulation** – the helper function is not exposed outside the method where it's used.
- **Readability** – the code remains close to where it's used.
- **Performance** – local functions can be more efficient than lambdas because they don't allocate a delegate in some cases.

Local functions can also be placed after the `return` statement (the compiler hoists them), but it's conventional to put them near the top or bottom of the containing method.

---

## 5.5 Expression‑Bodied Members

For very simple methods, properties, or other members, C# provides a concise syntax called **expression‑bodied members**. Instead of a block with braces, you use `=>` followed by an expression.

### Expression‑Bodied Methods

```csharp
public int Add(int a, int b) => a + b;

public void PrintMessage(string msg) => Console.WriteLine(msg);
```

If the method returns `void`, the expression must be a statement (like a method call). For non‑void, the expression must return a value of the correct type.

### Expression‑Bodied Properties

Properties can also be expression‑bodied:

```csharp
public class Person
{
    private string _name;
    
    public string Name
    {
        get => _name;
        set => _name = value;
    }
    
    // Read‑only computed property
    public string Greeting => $"Hello, {Name}!";
}
```

For read‑only properties, you can omit the `get` keyword entirely:

```csharp
public string Greeting => $"Hello, {Name}!";
```

### Expression‑Bodied Constructors and Finalizers

Less common, but you can also use expression bodies for constructors and finalizers:

```csharp
public class MyClass
{
    private int _value;
    public MyClass(int value) => _value = value;
    ~MyClass() => Console.WriteLine("Finalized");
}
```

Expression‑bodied members make code more compact and readable when the logic is a single expression. Use them judiciously – if the logic is complex, stick with a traditional block.

---

## 5.6 Putting It All Together: A Practical Example

Let's build a small utility class that demonstrates the various method concepts we've covered: overloading, `ref`, `out`, `params`, local functions, and expression‑bodied members.

```csharp
using System;

namespace MethodDemo
{
    public class StringUtils
    {
        // Overloaded methods for joining strings
        public string Join(string separator, params string[] strings)
        {
            return string.Join(separator, strings);
        }

        public string Join(char separator, params string[] strings)
        {
            return string.Join(separator.ToString(), strings);
        }

        // Expression‑bodied method
        public string Greet(string name) => $"Hello, {name}!";

        // Method with out parameter
        public bool TryParseToInt(string input, out int result)
        {
            return int.TryParse(input, out result);
        }

        // Method with ref parameter (swap)
        public void SwapStrings(ref string a, ref string b)
        {
            string temp = a;
            a = b;
            b = temp;
        }

        // Method demonstrating local function
        public int CountVowels(string input)
        {
            if (string.IsNullOrEmpty(input)) return 0;

            // Local function to check if a character is a vowel
            bool IsVowel(char c)
            {
                c = char.ToLower(c);
                return c == 'a' || c == 'e' || c == 'i' || c == 'o' || c == 'u';
            }

            int count = 0;
            foreach (char ch in input)
            {
                if (IsVowel(ch)) count++;
            }
            return count;
        }

        // Method using in parameter (performance with large structs, but here just for demo)
        public void DescribePoint(in Point p)
        {
            Console.WriteLine($"Point at ({p.X}, {p.Y})");
            // p.X = 10; // Would cause compiler error
        }
    }

    // Simple struct for demonstration
    public struct Point
    {
        public int X { get; set; }
        public int Y { get; set; }
    }

    class Program
    {
        static void Main()
        {
            StringUtils utils = new StringUtils();

            // Overloaded Join
            string result1 = utils.Join(", ", "apple", "banana", "cherry");
            Console.WriteLine(result1); // apple, banana, cherry

            string result2 = utils.Join('-', "one", "two", "three");
            Console.WriteLine(result2); // one-two-three

            // Expression‑bodied method
            Console.WriteLine(utils.Greet("Alice"));

            // Out parameter
            if (utils.TryParseToInt("123", out int number))
            {
                Console.WriteLine($"Parsed: {number}");
            }

            // Ref parameter (swap)
            string first = "first";
            string second = "second";
            Console.WriteLine($"Before swap: first = {first}, second = {second}");
            utils.SwapStrings(ref first, ref second);
            Console.WriteLine($"After swap: first = {first}, second = {second}");

            // Local function for vowel counting
            string sentence = "Hello World";
            int vowelCount = utils.CountVowels(sentence);
            Console.WriteLine($"Vowels in '{sentence}': {vowelCount}");

            // In parameter
            Point p = new Point { X = 10, Y = 20 };
            utils.DescribePoint(in p);
        }
    }
}
```

**Explanation:**

- **Overloaded `Join`** – one uses a string separator, another uses a char separator.
- **Expression‑bodied `Greet`** – simple one‑liner.
- **`TryParseToInt`** – demonstrates `out` parameter, wrapping `int.TryParse`.
- **`SwapStrings`** – `ref` parameters to swap two strings.
- **`CountVowels`** – uses a local function `IsVowel` to encapsulate the vowel check, keeping the main method clean.
- **`DescribePoint`** – shows `in` parameter (though for a small struct it's overkill, it illustrates the concept).

This example shows how different method features can work together in a real class.

---

## 5.7 Common Pitfalls and Best Practices

### 1. Naming Methods Clearly
Method names should be verbs or verb phrases that clearly indicate what the method does (e.g., `CalculateTotal`, `SaveData`, `GetUserName`). Avoid vague names like `DoStuff`.

### 2. Keep Methods Focused
A method should do one thing and do it well. If a method is long and does multiple tasks, consider splitting it into smaller helper methods. This improves readability, testability, and maintainability.

### 3. Limit the Number of Parameters
Too many parameters make a method hard to use and understand. If you find yourself passing many parameters, consider:
- Grouping related parameters into a class or struct.
- Using a parameter object pattern.
- Using optional parameters or overloading to provide simpler versions.

### 4. Be Consistent with Parameter Order
If you have multiple overloads or related methods, keep parameter order consistent (e.g., always put the most important or most frequently used parameters first).

### 5. Use `ref` and `out` Sparingly
Passing by reference can make code harder to follow because it's not obvious that a method modifies its arguments. Prefer returning a value or a tuple when you need multiple outputs. `out` is acceptable for the TryParse pattern.

### 6. Prefer `Try` Pattern for Methods That Can Fail
Instead of throwing exceptions for expected failures (like parsing), provide a `Try` method that returns a `bool` and an `out` parameter. This is more efficient and avoids exception overhead in common cases.

### 7. Validate Parameters
Check for invalid inputs at the beginning of a method and throw appropriate exceptions (like `ArgumentNullException`, `ArgumentException`). This is known as the "guard clause" pattern.

```csharp
public void ProcessData(string data)
{
    if (data == null) throw new ArgumentNullException(nameof(data));
    if (data.Length == 0) throw new ArgumentException("Data cannot be empty", nameof(data));
    // rest of method
}
```

### 8. Use Expression‑Bodied Members for Simplicity
If a method or property is a single expression, use the expression‑bodied syntax to reduce clutter. But if it's more complex, stick with a block.

### 9. Local Functions vs. Private Methods
Use local functions when the helper is only relevant to a single method and you want to keep it close. If the helper might be useful elsewhere, make it a private method.

### 10. Document Public Methods
For public APIs, consider adding XML comments to explain what the method does, its parameters, return value, and any exceptions it throws. This improves IntelliSense and helps other developers.

```csharp
/// <summary>
/// Calculates the sum of two integers.
/// </summary>
/// <param name="a">First integer.</param>
/// <param name="b">Second integer.</param>
/// <returns>The sum of a and b.</returns>
public int Add(int a, int b) => a + b;
```

---

## 5.8 Chapter Summary

In this chapter, you've gained a deep understanding of methods – the fundamental building blocks of C# programs:

- **Method signatures** define the name, parameters, and return type. Access modifiers control visibility.
- **Parameters** can be passed by value (default), by reference (`ref`), as output (`out`), or as read‑only reference (`in`). You also learned about `params` for variable numbers of arguments, and named/optional arguments for flexibility.
- **Method overloading** lets you create multiple methods with the same name but different parameter lists, providing convenience and clarity.
- **Local functions** allow you to define helper methods within another method, encapsulating logic without exposing it.
- **Expression‑bodied members** give you a concise syntax for simple methods, properties, and more.

Armed with these tools, you can now structure your code into reusable, self‑contained units. Good method design is key to writing clean, maintainable applications.

In the next chapter, **Object‑Oriented Programming (OOP) – The C# Way**, we'll take a major step forward. You'll learn how to model real‑world entities using classes and objects, and discover the core principles of OOP: encapsulation, inheritance, and polymorphism. This is where C# truly shines, and you'll start building more sophisticated and organised systems.

**Exercises:**

1. Write a method named `IsEven` that takes an integer and returns `true` if it's even. Write an overload that takes a string representation of a number and tries to parse it.
2. Create a method `Calculate` that takes two integers and an operation (as a string, e.g., "+", "-", "*", "/") and returns the result. Use `out` to indicate whether the operation was successful (e.g., division by zero).
3. Implement a `Swap` method that works for any type (using generics – we haven't covered generics formally, but try to use `ref` with `object`? Actually, better to wait for generics chapter. Instead, use `ref` with `int` and `string` overloads.)
4. Write a method `PrintStars` that uses `params int[]` to print a number of stars for each argument (e.g., `PrintStars(3, 5, 2)` prints three lines of stars with lengths 3, 5, 2).
5. Refactor a piece of code from a previous chapter to use local functions where appropriate.

Now, get ready for Chapter 6, where we'll dive into the heart of C# – classes and objects!